In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import datetime
import re
import json

In [2]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [3]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [4]:
resultadoAmsa = sf.query_all(
    """
        SELECT CALENDAR_YEAR(CreatedDate), COUNT(Id),CALENDAR_MONTH(CreatedDate)
    from Paciente_Programas__c
    Where Programas_por_Empresas__r.Holding__c like '%antofagasta minerals%'
    GROUP BY CALENDAR_YEAR(CreatedDate), CALENDAR_MONTH(CreatedDate)
    ORDER BY CALENDAR_MONTH(CreatedDate)
        """
)
amsaIngreso = pd.DataFrame(resultadoAmsa["records"])

amsaIngreso = amsaIngreso[
    [
        "expr0",
        "expr2",
        "expr1",
    ]
]
amsaIngreso = amsaIngreso.rename(
    columns={
        "expr0": "Año",
        "expr1": "Cantidad",
        "expr2": "Mes",
    }
)

meses = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}

amsaIngreso.replace({"Mes": meses}, inplace=True)

amsaIngreso.to_excel("F:/Reportes SF/Zurich/Cantida_AMSA.xlsx", index=False)
amsaIngreso.head(12)

,Año,Mes,Cantidad
0,2023,Enero,4091
1,2023,Febrero,267
2,2023,Marzo,195
3,2023,Mayo,3150
4,2023,Junio,346
5,2023,Julio,125
6,2023,Agosto,1
7,2023,Septiembre,174
8,2023,Octubre,200
9,2022,Noviembre,3


In [5]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false  
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(8060, 8)

In [6]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7746, 5)

In [7]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

(8066, 10)

In [8]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

fin empresas y comienzo de rescate de pacientes


In [9]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True 
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1880756, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


In [10]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

(1500901, 4)

In [11]:
# Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

(1500901, 13)

In [12]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [ ]:
# pacientes_empresas22 = pd.merge(
# left=Pacientes_SF,
# right=empresas_unidas,
# how="left",
# left_on="Id Account Paciente",
# right_on="Id Account",
# )
# pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
# pacientes_empresas22 = pacientes_empresas22[
# ["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
# ]

In [ ]:
# pacientes_empresas22.shape

In [ ]:
# apprix_1 = pacientes_empresas22.iloc[:848826,:]

# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)

In [ ]:
# apprix_2 = pacientes_empresas22.iloc[848827:,:]#
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [ ]:
# pacientes_empresas22.head()

Carga de Archivo carga


In [167]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Vida Security"
Pais = "Chile"
# Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion

In [168]:
Pacientes_Archivo.head(10)

,Cog Convenio,Nombre Convenio,poliza,Rut Contratante,Dv Contratante,Unnamed: 5,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,...,Dv Asegurado,rut1,Parentesco,nombres,apellidos,nacimiento,sexo,Inicio Vigencia,Termino Vigencia,email
0,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,6,10045133-6,TITULAR,FERNANDO,SAAVEDRA CID,1967-04-18,M,2023-01-01 00:00:00,2023-12-31 00:00:00,FERNANDO.SAAVEDRA@CL.ABB.COM
1,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,6,11559451-6,CÓNYUGE,PATRICIA,NAVARRO GAJARDO,1970-04-18,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN
2,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,9,20182827-9,HIJO,FERNANDO JAVIER,SAAVEDRA NAVARRO,1999-05-31,M,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN
3,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,4,20657341-4,HIJO,ANAIS BELEN,SAAVEDRA NAVARRO,2001-01-16,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN
4,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,1,10078600-1,TITULAR,JUSTINIANO CELESTINO,GUTIERREZ LILLO,1965-09-18,M,2023-01-01 00:00:00,2023-12-31 00:00:00,TITOGUTIERREZLILLO@GMAIL.COM
5,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,K,10573770-K,CÓNYUGE,MARINA ALEJANDRA,TORRES IBAÑEZ,1967-02-16,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN
6,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,8,10131170-8,TITULAR,LUIS,FIGUEROA GUZMAN,1964-09-26,M,2023-01-01 00:00:00,2023-12-31 00:00:00,LUIS.FIGUEROA@CL.ABB.COM
7,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,5,9678768-5,CÓNYUGE,CECILIA,SANCHEZ GALLEGUILLOS,1964-07-07,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN
8,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,7,10162539-7,TITULAR,EDMUNDO,HIDALGO BECERRA,1967-10-18,M,2023-01-01 00:00:00,2023-12-31 00:00:00,EDMUNDO.HIDALGO-BECERRA@CL.ABB.COM
9,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,6,10492321-6,CÓNYUGE,CLAUDIA PAZ,PEREZ CERON,1970-02-02,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN


In [169]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)

In [170]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,Cog Convenio,Nombre Convenio,poliza,Rut Contratante,Dv Contratante,Unnamed: 5,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,...,rut1,Parentesco,nombres,apellidos,nacimiento,sexo,Inicio Vigencia,Termino Vigencia,email,rut
0,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,10045133-6,TITULAR,FERNANDO,SAAVEDRA CID,1967-04-18,M,2023-01-01 00:00:00,2023-12-31 00:00:00,FERNANDO.SAAVEDRA@CL.ABB.COM,100451336
1,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,11559451-6,CÓNYUGE,PATRICIA,NAVARRO GAJARDO,1970-04-18,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN,115594516
2,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,20182827-9,HIJO,FERNANDO JAVIER,SAAVEDRA NAVARRO,1999-05-31,M,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN,201828279
3,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,20657341-4,HIJO,ANAIS BELEN,SAAVEDRA NAVARRO,2001-01-16,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN,206573414
4,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,10078600-1,TITULAR,JUSTINIANO CELESTINO,GUTIERREZ LILLO,1965-09-18,M,2023-01-01 00:00:00,2023-12-31 00:00:00,TITOGUTIERREZLILLO@GMAIL.COM,100786001


In [171]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_5716\1949753364.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,Cog Convenio,Nombre Convenio,poliza,Rut Contratante,Dv Contratante,Unnamed: 5,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,...,rut1,Parentesco,nombres,apellidos,nacimiento,sexo,Inicio Vigencia,Termino Vigencia,email,rut
0,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,10045133-6,TITULAR,FERNANDO,SAAVEDRA CID,1967-04-18,M,2023-01-01 00:00:00,2023-12-31 00:00:00,FERNANDO.SAAVEDRA@CL.ABB.COM,100451336
1,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,11559451-6,CÓNYUGE,PATRICIA,NAVARRO GAJARDO,1970-04-18,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN,115594516
2,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,20182827-9,HIJO,FERNANDO JAVIER,SAAVEDRA NAVARRO,1999-05-31,M,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN,201828279
3,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,20657341-4,HIJO,ANAIS BELEN,SAAVEDRA NAVARRO,2001-01-16,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN,206573414
4,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,10078600-1,TITULAR,JUSTINIANO CELESTINO,GUTIERREZ LILLO,1965-09-18,M,2023-01-01 00:00:00,2023-12-31 00:00:00,TITOGUTIERREZLILLO@GMAIL.COM,100786001


In [172]:
def siPrimerCero(y):
    if y == "00":
        return "Nones"
    if y[0] == "0":
        y = y[1:]
    return y

In [173]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].apply(lambda y: siPrimerCero(y))

In [174]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].apply(lambda y: siPrimerCero(y))

Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_5716\620742920.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_5716\620742920.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [175]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(16122, 36)

In [176]:
Pacientes_con_id.head()

,Cog Convenio,Nombre Convenio,poliza,Rut Contratante,Dv Contratante,Unnamed: 5,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,True,0018c00002bHe9DAAS,100451336,Fernando Saavedra Cid,RUT,None,None,None,Hombre,None
1,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,True,0018c00002bKLVXAA4,115594516,Patricia Navarro Gajardo,RUT,None,None,None,Mujer,None
2,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,True,0018c00002bHeJTAA0,201828279,Fernando Javier Saavedra Navarro,RUT,None,None,None,Hombre,None
3,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,True,0018c00002bJoFLAA0,206573414,Anais Belen Saavedra Navarro,RUT,None,None,None,Mujer,None
4,96,CARE ASSISTANCE Plan BASICO,281988,92805000,9,NaN,ABB S A,92805000,9,ABB S A,...,True,0018c00002bIWU6AAO,100786001,Justiniano Celestino Gutierrez Lillo,RUT,None,None,None,Hombre,None


In [177]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")


def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""

In [178]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = Pacientes_con_id["email"].astype(str).str.lower()
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(
        lambda y: isValid(y)
    )
    # Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "H": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "Prefiero no responder",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "M": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon": "Hombre",
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("/", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("null", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NULL", "")


Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
254,False,Raul Rodrigo,Araya Ortiz,13261685K,,1977-08-04,M,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
255,False,Vanessa Alejandra,Gajardo Nuñez,155636734,,1983-03-26,Mujer,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
256,False,Lucas Alexander,Araya Gajardo,245413580,,2014-02-19,M,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
257,False,Raul Reinaldo,Araya Gajardo,210054936,,2002-04-26,M,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
258,False,Bruno Alonso,Araya Gajardo,252246525,,2015-12-14,M,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [179]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña
""" cha_arc = datetime.now()
dia_hoy = str(fecha_arc.day)
mes_hoy = str(fecha_arc.month)
 """


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "Sura": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
# clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)
""" clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)
 """

' clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)\n '

In [180]:
""" print(dia_hoy) """

' print(dia_hoy) '

In [181]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
385545,a1Y8c00000CldqXEAR,0018c00002azlRDAAY,a1W8c0000083rYiEAI,Vida Security,0018c00002axQfVAAU,S.Q.M. Industrial S.A.,Grupo S.Q.M.,Vida Security,S.Q.M. Industrial S.A.,283654,79947100-0,EMP6158,A-9176
386393,a1Y8c00000CleLsEAJ,0018c00002azmGuAAI,a1W8c0000083rYiEAI,Vida Security,0018c00002axQfVAAU,S.Q.M. Industrial S.A.,Grupo S.Q.M.,Vida Security,S.Q.M. Industrial S.A.,283654,79947100-0,EMP6158,A-9176
449181,a1Y8c00000Cm2JJEAZ,0018c00002b0RUiAAM,a1W8c0000083rZsEAI,Vida Security,0018c00002axQfOAAU,Minera Escondida Ltda.,Grupo BHP,Vida Security,Minera Escondida Ltda.,282349,79587210-8,EMP6151,A-9244
465134,a1Y8c00000CmDFMEA3,0018c00002bHw4eAAC,a1W8c0000083rZiEAI,Vida Security,0018c00002axQfKAAU,Empresa De Transporte Ferroviario S.A.,Ferronor,Vida Security,Empresa De Transporte Ferroviario S.A.,284254,96545600-7,EMP6147,A-9238
470999,a1Y8c00000CmGvvEAF,0018c00002bI3ErAAK,a1W8c0000083rYcEAI,Vida Security,0018c00002axQfEAAU,Emaresa Ingenieros Y Representaciones S.A.,Emaresa Ingenieros Y Representaciones S.A.,Vida Security,Emaresa Ingenieros Y Representaciones S.A.,283047,83162400-0,EMP6141,A-9170


In [182]:
campaña_seleccionada.shape

(119605, 13)

In [183]:
""" f campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )
 """

' f campaña != "MetLife":\n    pacientes_en_campaña = pd.merge(\n        left=campaña_seleccionada,\n        right=Pacientes_con_id,\n        how="left",\n        left_on="Id Account",\n        right_on="Id Account Paciente",\n    )\n    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"\n    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"\n    pacientes_en_campaña.to_excel(\n        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False\n    )\n    campaña_en_pacientes = pd.merge(\n        left=Pacientes_con_id,\n        right=campaña_seleccionada,\n        how="left",\n        left_on="Id Account Paciente",\n        right_on="Id Account",\n    )\n    campaña_en_pacientes.to_excel(\n        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False\n    )\n '

In [184]:
""" if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    ) """

' if campaña != "MetLife1":\n    campaña_en_pacientes = pd.merge(\n        left=Pacientes_con_id,\n        right=campaña_seleccionada,\n        how="left",\n        left_on="Id Account Paciente",\n        right_on="Id Account",\n    )\n    if ("poliza" in campaña_en_pacientes.columns) == True:\n        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)\n\n    campaña_en_pacientes = campaña_en_pacientes.rename(\n        columns={"Programas_por_Empresas__c": "programas por empresa"}\n    )\n    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]\n    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[\n        "programas por empresa"\n    ]\n    campaña_en_pacientes.to_excel(\n        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False\n    ) '

In [185]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [186]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)